In [1]:
import pandas as pd
import datetime as dt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

In [2]:
df1 = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [3]:
len(df1.columns)

19

## Q2. Computing duration

In [4]:
df1['duration'] = df1['tpep_dropoff_datetime']-df1['tpep_pickup_datetime']
df1['duration'] = df1['duration'].dt.total_seconds()/60

In [5]:
df1

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [6]:
df1['duration'].std()

42.59435124195458

## Q3. Dropping outliers

In [7]:
df1_org = df1.copy()
df1 = df1[(df1['duration'] >= 1) & (df1['duration'] <=60 )]
len(df1)/len(df1_org)

0.9812202822125979

## Q4. One-hot encoding

In [8]:
cols = ['PULocationID', 'DOLocationID']
df1[cols].isnull().sum()

PULocationID    0
DOLocationID    0
dtype: int64

In [9]:
df1[cols] = df1[cols].astype('str')

C:\Users\ipek.guler\AppData\Local\Temp\ipykernel_12976\1818632783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[cols] = df1[cols].astype('str')


In [10]:
train_dicts = df1[cols].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(3009173, 515)

## Q5. Training a model

In [11]:
y_train = df1['duration'].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261931416412

## Q6. Evaluating the model

In [13]:
df2['duration'] = df2['tpep_dropoff_datetime']-df2['tpep_pickup_datetime']
df2['duration'] = df2['duration'].dt.total_seconds()/60
df2 = df2[(df2['duration'] >= 1) & (df2['duration'] <=60 )]
df2[cols].isnull().sum()

PULocationID    0
DOLocationID    0
dtype: int64

In [14]:
df2[cols] = df2[cols].astype('str')

In [15]:
val_dicts = df2[cols].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df2['duration'].values

In [16]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.8118162035401735